In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
#运行次数
max_steps = 1001
#图片数量
image_num = 3000
#文件路径
DIR = "F:/Documents_of_Tensorflow/"

#定义会话
sess = tf.Session()

#载入图片, mnist.test.images[:image_num]指打包从第0张到第image_num张
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

'''
#参数概要
def variableSummaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        mean_scaler = tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        stddev_scaler = tf.summary.scalar('stddev',stddev)#标准差
        max_scaler = tf.summary.scalar('max',tf.reduce_max(var))#最大值
        min_scaler = tf.summary.scalar('min',tf.reduce_min(var))#最小值
        histogram_scaler = tf.summary.histogram('histogram',var)#直方图
'''
#命名空间
with tf.name_scope('input'):
    #定义两个placeholder, 这里的none表示第一个维度可以是任意的长度
    x = tf.placeholder(tf.float32, [None,784],name="x-input")
    #正确的标签
    y = tf.placeholder(tf.float32, [None,10],name="y-input")

#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1]) #第四个参数1指的是黑白图像
                                                # 若是彩色图片则是3
    image_scaler = tf.summary.image('input',image_shaped_input,10)

with tf.name_scope('layer'):
    #创建一个简单的神经网络
    with tf.name_scope('wights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        #variableSummaries(W)
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(W)
            meanW_scaler = tf.summary.scalar('mean',mean)#平均值
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(W-mean)))
            stddevW_scaler = tf.summary.scalar('stddev',stddev)#标准差
            maxW_scaler = tf.summary.scalar('max',tf.reduce_max(W))#最大值
            minW_scaler = tf.summary.scalar('min',tf.reduce_min(W))#最小值
            histogramW_scaler = tf.summary.histogram('histogram',W)#直方图
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        #variableSummaries(b)
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(b)
            meanb_scaler = tf.summary.scalar('mean',mean)#平均值
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(b-mean)))
            stddevb_scaler = tf.summary.scalar('stddev',stddev)#标准差
            maxb_scaler = tf.summary.scalar('max',tf.reduce_max(b))#最大值
            minb_scaler = tf.summary.scalar('min',tf.reduce_min(b))#最小值
            histogramb_scaler = tf.summary.histogram('histogram',b)#直方图       
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W)+b  # matmul是矩阵相乘
    with tf.name_scope('softmax'):    # softmax激活函数
        prediction = tf.nn.softmax(wx_plus_b)

#二次代价函数
with tf.name_scope('loss'):
    #交叉熵代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
    loss_scaler = tf.summary.scalar('loss',loss)
    
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

#初始化变量
sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个boolen型列表中
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction, 1))#argmax返回一维张量中最大值所在位置
    with tf.name_scope('accuracy'):
        #求准确度,把correct_prediction变为float32类型
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        accuracy_scaler = tf.summary.scalar('accuracy',accuracy)

#产生metadata文件
if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
with open(DIR+'projector/projector/metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))   #函数argmax求出最大值所在的位置，即标签
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')

#合并所有的summary      报错的地方！！！
#merged = tf.summary.merge_all()  # 报错的地方！！！

projector_writer = tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR + 'projector/data/mnist_10K_sprite.png'
embed.sprite.single_image_dim.extend([28,28])  # 按照28*28大小切分图片为单个数字图片
projector.visualize_embeddings(projector_writer,config)

for i in range(max_steps):
    #每个批次100个样本
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    #summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    meanW_metall,stddevW_metall,maxW_metall,minW_metall,histogramW_metall,meanb_metall,stddevb_metall,maxb_metall,minb_metall,histogramb_metall,loss_metall,accuracy_metall,image_metall,_ = sess.run([meanW_scaler,stddevW_scaler,maxW_scaler,minW_scaler,histogramW_scaler,meanb_scaler,stddevb_scaler,maxb_scaler,minb_scaler,histogramb_scaler,loss_scaler,accuracy_scaler,image_scaler,train_step], feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    
    projector_writer.add_summary(meanW_metall,i)
    projector_writer.add_summary(stddevW_metall,i)
    projector_writer.add_summary(maxW_metall,i)
    projector_writer.add_summary(minW_metall,i)
    projector_writer.add_summary(histogramW_metall,i)
    projector_writer.add_summary(meanb_metall,i)
    projector_writer.add_summary(stddevb_metall,i)
    projector_writer.add_summary(maxb_metall,i)
    projector_writer.add_summary(minb_metall,i)
    projector_writer.add_summary(histogramb_metall,i)
    projector_writer.add_summary(loss_metall,i)
    projector_writer.add_summary(accuracy_metall,i)
    projector_writer.add_summary(image_metall,i)
    
    if i%100 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter"+str(i)+",Testing Accuracy"+str(acc))

# 保存训练好的模型
saver.save(sess,DIR+'projector/projector/a_model.ckpt',global_step=max_steps)
projector_writer.close()
sess.close()



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Iter0,Testing Accuracy0.0982
Iter100,Testing Accuracy0.5998
Iter200,Testing Accuracy0.7519
Iter300,Testing Accuracy0.7938
Iter400,Testing Accuracy0.8029
Iter500,Testing Accuracy0.8089
Iter600,Testing Accuracy0.8111
Iter700,Testing Accuracy0.8167
Iter800,Testing Accuracy0.8192
Iter900,Testing Accuracy